In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [4]:
content = pd.read_csv('content.csv')
topics = pd.read_csv('topics.csv', keep_default_na=False)
correlations = pd.read_csv('correlations.csv', converters={'content_ids': lambda x: x[1:-1].split(' ')})
# only required to make predictions for 5 topics in the final submission, select those 5 topics based on the sample_submission.csv
final_topics = topics[topics.id.isin(['t_00004da3a1b2', 't_00068291e9a4', 't_00069b63a70a', 't_0006d41a73a8', 't_4054df11a74e'])]

# correlations.csv stores content_ids as a list for each topic, explode so each row represents one topic-content match.
correlations = correlations.explode('content_ids', ignore_index=True)
all_data = correlations.merge(content, left_on='content_ids', right_on='id').drop(columns='id')
y = all_data['topic_id']
X = all_data.drop(columns='topic_id')
print(y.shape)
print(X.shape)

(170322,)
(170322, 8)


In [19]:
# Print topic heiarchy 
def print_topic_heiarchy(topic):
    topic_level = topics[topics.id==topic].level.values[0]
    while topic_level != '':
        topic_level = topics[topics.id==topic].level.values[0]
        print(topics[['id', 'title']][topics.id==topic])
        parent = topics[topics.id==topic].parent.values[0]
        topic = parent
        
        
        
        print_topic_heiarchy(topic)


print_topic_heiarchy('t_00068291e9a4')
# print(topics.iloc[49677])
# print(topics[topics.id=='t_04b58233425c'])

               id                            title
2  t_00068291e9a4  Entradas e saídas de uma função
                   id             title
62628  t_d14b6c2a2b70  Álgebra: funções
                   id   title
20817  t_461c9a8dd863  9º Ano
                   id                                 title
49677  t_a6420bcff569  Matemática por ano (Alinhada à BNCC)
                  id                              title
1374  t_04b58233425c  Khan Academy (Português (Brasil))


IndexError: index 0 is out of bounds for axis 0 with size 0

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
